# Image Enhancement Pipeline: Zero-DCE + Real-ESRGAN + GFPGAN

This notebook processes a single input image through three models:
1. Zero-DCE for low-light enhancement
2. Real-ESRGAN for super-resolution
3. GFPGAN for face restoration

You can upload an image, and it will be processed step-by-step, with output shown at each stage.

In [ ]:
# Install dependencies
!pip install basicsr facexlib gfpgan realesrgan -q
!pip install -U torch torchvision --extra-index-url https://download.pytorch.org/whl/cu113 -q

# Install Zero-DCE implementation (lightweight repo)
!pip install git+https://github.com/Li-Chongyi/Zero-DCE.git -q

In [ ]:
# Imports
import cv2
import torch
import numpy as np
from PIL import Image
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

from Zero_DCE import zero_dce  # zero dce repo import
from realesrgan import RealESRGAN
from gfpgan import GFPGANer
import os

# Utility functions
def load_image_cv2(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def show_img(img, title=None):
    plt.figure(figsize=(10,10))
    plt.axis('off')
    if title is not None:
        plt.title(title)
    plt.imshow(img)
    plt.show()


## Upload your image

In [ ]:
# Upload an input image
from google.colab import files
uploaded = files.upload()
input_image_path = list(uploaded.keys())[0]
print(f"Uploaded file: {input_image_path}")

# Load and show input image
img = load_image_cv2(input_image_path)
show_img(img, 'Input Image')

## Step 1: Low-Light Enhancement with Zero-DCE

In [ ]:
# Zero-DCE expects images normalized [0,1]
img_float = img.astype(np.float32)/255.
# Apply zero-dce enhancement
enhanced = zero_dce.Inference(img_float)
enhanced = np.clip(enhanced*255,0,255).astype(np.uint8)
show_img(enhanced, 'Zero-DCE Enhanced Image')

# Save enhanced image for next steps
cv2.imwrite('enhanced.png', cv2.cvtColor(enhanced, cv2.COLOR_RGB2BGR))

## Step 2: Super-Resolution with Real-ESRGAN

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

# Initialize Real-ESRGAN model
model = RealESRGAN(device, scale=4)  # 4x upscaling
model.load_weights("https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.0/RealESRGAN_x4plus.pth")

# Load enhanced image
img_enhanced = Image.open('enhanced.png').convert('RGB')

# Apply super resolution
sr_img = model.predict(img_enhanced)

# Convert to numpy and show
sr_img_np = np.array(sr_img)
show_img(sr_img_np, 'Real-ESRGAN Super Resolved Image')

# Save for next step
sr_img.save('sr.png')

## Step 3: Face Restoration with GFPGAN

In [ ]:
from gfpgan import GFPGANer

# Load GFPGAN model
gfpganer = GFPGANer(
    model_path="https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth",
    upscale=1,
    arch='clean',
    channel_multiplier=2,
    bg_upsampler=None,
    device=device
)

# Load super-resolved image
sr_img = cv2.imread('sr.png', cv2.IMREAD_COLOR)

# GFPGAN expects BGR input
cropped_faces, restored_faces, restored_img = gfpganer.enhance(
    sr_img,
    has_aligned=False,
    only_center_face=False,
    paste_back=True
)

# Show final image
restored_img_rgb = cv2.cvtColor(restored_img, cv2.COLOR_BGR2RGB)
show_img(restored_img_rgb, 'Final GFPGAN Restored Image')

# Save final image
cv2.imwrite('final_restored.png', restored_img)
print("Processing complete. Final image saved as final_restored.png")